In [1]:
import sqlalchemy
import pandas as pd
import datetime
import json
import glob
import os
dict_config=json.load(open("/mnt/clients/juba/hqjubaapp02/jliang/Projects/Big_Lots/Predictive_Model/Model_Scripts/config.json"))
high_date=dict_config['pos_end_date']
username=dict_config['username']
password=dict_config['password']
database=dict_config['database']

In [2]:
sql_engine=sqlalchemy.create_engine("mysql+pymysql://%s:%s@localhost/%s" % (username, password, database))

In [4]:
pd.options.display.max_columns=400

In [5]:
pd.read_sql("show tables;",con=sql_engine)

,Tables_in_BigLots
0,BL_CCPA_email
1,BL_POS_Item
2,BL_Rewards_Master
3,NEall_id_from_crm_pos_until_20210102
4,NEall_id_pos_until_20201205
5,NEall_id_pos_until_20201212
6,NEall_id_pos_until_20201219
7,NEall_id_pos_until_20201226
8,NEall_id_pos_until_20210102
9,Pred_POS_Department


In [6]:
pd.read_sql("desc Pred_POS_Department;",con=sql_engine)

,Field,Type,Null,Key,Default,Extra
0,location_id,int(11),YES,MUL,None,
1,transaction_dt,date,YES,MUL,None,
2,transaction_id,varchar(16),YES,,None,
3,customer_id_hashed,char(64),YES,MUL,None,
4,department_id,varchar(16),YES,MUL,None,
5,sales,"decimal(10,2)",YES,,None,
6,units,int(11),YES,,None,
7,trans_order_since_18Q1,int(11),YES,MUL,None,


In [16]:
df_qc=pd.read_sql("select * from Pred_POS_Department where transaction_dt='2021-01-01' limit 200;",con=sql_engine)
list_qc_ids=df_qc['customer_id_hashed'].unique().tolist()

In [18]:
i=11

str_id="'"+list_qc_ids[i]+"'"
str_id

"'0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2e89e8c4fbd5e37f9d9'"

In [20]:
df_res_id=pd.read_sql("select * from Pred_POS_Department where customer_id_hashed=%s;"%str_id,con=sql_engine)
df_res_id.tail(20)


,location_id,transaction_dt,transaction_id,customer_id_hashed,department_id,sales,units,trans_order_since_18Q1
90,1372,2020-07-25,6249,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,130,2.55,1,18
91,1372,2020-07-25,6249,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,352,23.25,4,18
92,1372,2020-07-25,6249,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,364,40.00,4,18
93,1372,2020-07-25,6249,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,410,1.00,1,18
94,1372,2020-07-25,6249,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,470,10.00,1,18
95,1372,2020-09-15,7175,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,612,299.99,2,19
96,1372,2020-10-24,9866,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,108,3.00,2,20
97,1372,2020-10-24,9866,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,150,25.69,3,20
98,1372,2020-10-24,9866,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,410,3.00,2,20
99,1372,2020-12-27,4765,0003f80dfebab5cb721df4fb59c5a0b3a3b10ba5fc10e2...,350,75.00,2,21


In [21]:
df_crm=pd.read_sql("select * from BL_Rewards_Master limit 20;",con=sql_engine)
df_crm

,customer_id_hashed,email_address_hash,sign_up_date,sign_up_channel,sign_up_location,customer_zip_code,transaction_count,transaction_amount,experian_multi_cluster,experian_demo_cluster,purchase_channel,email_unsubscribe_indicator,email_undeliverable_indicator,file_path
0,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,4a1f9ca814426d0168b4ee9ac037be04151dff5631d78a...,2016-02-10,STORE,5309.0,36081,1.0,32.80,0,0,STORES,N,N,/home/jian/Projects/Big_Lots/Loyal_members/loy...
1,000001dadc0265bf9d250566d74e0006323f18b5826641...,26454de1f315ee0c13c8c9033f372f50a3f02a16691849...,2018-09-22,STORE,4061.0,93722,NaN,NaN,None,None,None,N,N,/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Ye...
2,000002c86b9805b916f2a04b7a0031bb9ee03224e98221...,4765e094d9c921485df3befed517927f7119980982e000...,2014-12-01,ONLINE,NaN,22560,1.0,61.73,6,2,STORES,N,N,/home/jian/Projects/Big_Lots/Loyal_members/loy...
3,000003066bafc6b19c55d3be5fc646a253c659e9e6baea...,d461fd1bb5f270aaa66016cc60bf6aa16a521565360e41...,2013-02-28,ONLINE,0.0,21229,4.0,127.05,0,0,STORES,N,N,/home/jian/Projects/Big_Lots/Loyal_members/loy...
4,0000034fec3c2cbbf7a2b38fbf80d84abc72d8bed269c4...,21620c4342292086ec48d349207620e54a528273c7ad3d...,2016-07-09,STORE,1852.0,14094,1.0,128.76,0,0,STORES,N,N,/home/jian/Projects/Big_Lots/Loyal_members/loy...
5,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,e7b239d884e48cdd957205e0ffb8e793bb7b8a391dc102...,2011-11-22,STORE,5116.0,21229,2.0,18.70,1,1,STORES,N,N,/home/jian/Projects/Big_Lots/Loyal_members/loy...
6,000004f194afdc2c2c26365032ea084974396ee79a7d37...,0ccd11d3b25023903ea6ed1c9a0e6d44c2965d55eaf8e2...,2019-05-16,ONLINE,6990.0,None,NaN,NaN,None,None,None,N,N,/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Ye...
7,000006784b07050cf4d45cfd14f59bae5a893a9e611752...,8837fedc09235920addd79fabe36aa7a09222ef1b1dfd9...,2016-02-10,STORE,5163.0,72015,2.0,35.00,3,4,STORES,N,Y,/home/jian/Projects/Big_Lots/Loyal_members/loy...
8,000006aa196c7510ca3b4617e7e52d9527e0190e9152fb...,4561dac9792c2a32bf05ed6fdc8fd29420c8967107c02b...,2010-04-06,ONLINE,0.0,80013,5.0,201.77,0,0,STORES,N,N,/home/jian/Projects/Big_Lots/Loyal_members/loy...
9,000006e85aba854de9992437786bddfbd409ddbebeeb80...,489892db7edde1567b9d5fc39e05a31ccaaef1d245a464...,2010-02-19,STORE,819.0,14527,3.0,39.00,3,2,STORES,N,N,/home/jian/Projects/Big_Lots/Loyal_members/loy...


In [22]:
df_pred_2_1=pd.read_sql("select * from all_NEall_id_pred_pos_2_1_pos_until_20201226 limit 20;",con=sql_engine)
df_pred_2_1


,id,weeks_since_sign_up,total_trans_since_registration,total_items,total_sales,trans_in_store,unique_stores,trans_online,department_minus_one_trans,department_108_trans,department_109_trans,department_110_trans,department_111_trans,department_114_trans,department_115_trans,department_120_trans,department_130_trans,department_140_trans,department_150_trans,department_160_trans,department_170_trans,department_210_trans,department_230_trans,department_250_trans,department_270_trans,department_310_trans,department_320_trans,department_330_trans,department_340_trans,department_350_trans,department_351_trans,department_352_trans,department_353_trans,department_354_trans,department_355_trans,department_360_trans,department_361_trans,department_362_trans,department_363_trans,department_364_trans,department_365_trans,department_366_trans,department_367_trans,department_370_trans,department_410_trans,department_425_trans,department_470_trans,department_480_trans,department_510_trans,department_520_trans,department_521_trans,department_526_trans,department_530_trans,department_540_trans,department_550_trans,department_560_trans,department_608_trans,department_610_trans,department_612_trans,department_615_trans,department_710_trans,department_800_trans
0,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,254,4,39.0,252.92,4,4,1,1,2,1,3,2,1,1,1,2,2,2,2,2,1,1,1,2,1,1,3,3,1,1,3,1,3,1,2,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,000001dadc0265bf9d250566d74e0006323f18b5826641...,118,7,54.0,293.95,7,2,1,1,3,1,3,1,1,1,3,2,3,1,2,2,1,1,1,2,1,1,1,2,1,1,2,1,2,1,5,1,1,4,1,1,1,2,1,3,1,2,1,1,1,1,1,1,1,2,1,1,1,1,1,7,1
2,0000023c8c46f19f7649b56583c6ffbfd530166465169b...,19,1,8.0,53.54,1,1,1,1,2,1,2,1,1,1,1,1,1,1,1,2,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,000003066bafc6b19c55d3be5fc646a253c659e9e6baea...,408,4,47.0,211.80,4,2,1,1,3,1,2,1,2,1,1,3,1,1,1,1,1,2,1,2,1,1,1,1,1,1,1,1,2,1,4,1,1,1,1,1,1,2,1,1,1,1,2,2,1,1,1,1,1,2,1,1,1,1,1,1,1
4,000003fd525b4a006e9cb82489c6059f12a0adfb46f4d7...,5,1,1.0,1.69,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,474,41,193.0,712.74,41,1,1,1,6,4,15,9,2,1,15,7,9,16,1,3,5,1,1,1,4,1,2,1,1,1,4,1,2,1,8,1,1,1,1,1,1,1,1,1,1,2,1,9,1,1,1,1,1,3,1,1,3,1,1,1,1
6,000004f194afdc2c2c26365032ea084974396ee79a7d37...,84,1,2.0,9.50,1,1,1,1,1,1,1,1,1,1,2,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,000006aa196c7510ca3b4617e7e52d9527e0190e9152fb...,559,21,190.0,876.25,21,2,1,1,6,1,8,3,4,1,12,2,3,2,2,1,1,4,1,1,2,2,5,2,2,4,2,1,2,1,4,1,2,1,2,1,1,1,1,6,1,3,4,7,2,1,1,1,1,5,1,1,1,1,1,1,1
8,000006e85aba854de9992437786bddfbd409ddbebeeb80...,566,3,13.0,53.45,3,2,1,1,2,1,1,1,1,1,2,2,1,2,1,1,2,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1
9,000006ff76d207f0c8fbb0262fb87c64b31a75dc89fd07...,123,7,42.0,282.70,7,1,1,1,1,2,3,4,1,1,1,1,1,3,4,2,1,1,1,1,1,3,1,1,1,2,1,1,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1


In [25]:
df_pred_2_2=pd.read_sql("select * from all_NEall_id_pred_pos_2_2_pos_until_20201226 limit 40;",con=sql_engine)
df_pred_2_2

,id,week_1st_trans,purchase_channel_1st_trans,total_sales_1st_trans,total_units_1st_trans,department_minus_one_1st_trans,department_108_1st_trans,department_109_1st_trans,department_110_1st_trans,department_111_1st_trans,department_114_1st_trans,department_115_1st_trans,department_120_1st_trans,department_130_1st_trans,department_140_1st_trans,department_150_1st_trans,department_160_1st_trans,department_170_1st_trans,department_210_1st_trans,department_230_1st_trans,department_250_1st_trans,department_270_1st_trans,department_310_1st_trans,department_320_1st_trans,department_330_1st_trans,department_340_1st_trans,department_350_1st_trans,department_351_1st_trans,department_352_1st_trans,department_353_1st_trans,department_354_1st_trans,department_355_1st_trans,department_360_1st_trans,department_361_1st_trans,department_362_1st_trans,department_363_1st_trans,department_364_1st_trans,department_365_1st_trans,department_366_1st_trans,department_367_1st_trans,department_370_1st_trans,department_410_1st_trans,department_425_1st_trans,department_470_1st_trans,department_480_1st_trans,department_510_1st_trans,department_520_1st_trans,department_521_1st_trans,department_526_1st_trans,department_530_1st_trans,department_540_1st_trans,department_550_1st_trans,department_560_1st_trans,department_608_1st_trans,department_610_1st_trans,department_612_1st_trans,department_615_1st_trans,department_710_1st_trans,department_800_1st_trans,week_recent_one_trans,week_counts_to_now_recent_one,recent_one_trans_also_1st,purchase_channel_1st_trans_recent_one,total_sales_recent_one_trans,total_units_recent_one_trans,department_minus_one_recent_one,department_108_recent_one,department_109_recent_one,department_110_recent_one,department_111_recent_one,department_114_recent_one,department_115_recent_one,department_120_recent_one,department_130_recent_one,department_140_recent_one,department_150_recent_one,department_160_recent_one,department_170_recent_one,department_210_recent_one,department_230_recent_one,department_250_recent_one,department_270_recent_one,department_310_recent_one,department_320_recent_one,department_330_recent_one,department_340_recent_one,department_350_recent_one,department_351_recent_one,department_352_recent_one,department_353_recent_one,department_354_recent_one,department_355_recent_one,department_360_recent_one,department_361_recent_one,department_362_recent_one,department_363_recent_one,department_364_recent_one,department_365_recent_one,department_366_recent_one,department_367_recent_one,department_370_recent_one,department_410_recent_one,department_425_recent_one,department_470_recent_one,department_480_recent_one,department_510_recent_one,department_520_recent_one,department_521_recent_one,department_526_recent_one,department_530_recent_one,department_540_recent_one,department_550_recent_one,department_560_recent_one,department_608_recent_one,department_610_recent_one,department_612_recent_one,department_615_recent_one,department_710_recent_one,department_800_recent_one,week_recent_two_trans,week_counts_to_now_recent_two,recent_two_trans_also_1st,purchase_channel_1st_trans_recent_two,total_sales_recent_two_trans,total_units_recent_two_trans,department_minus_one_recent_two,department_108_recent_two,department_109_recent_two,department_110_recent_two,department_111_recent_two,department_114_recent_two,department_115_recent_two,department_120_recent_two,department_130_recent_two,department_140_recent_two,department_150_recent_two,department_160_recent_two,department_170_recent_two,department_210_recent_two,department_230_recent_two,department_250_recent_two,department_270_recent_two,department_310_recent_two,department_320_recent_two,department_330_recent_two,department_340_recent_two,department_350_recent_two,department_351_recent_two,department_352_recent_two,department_353_recent_two,department_354_recent_two,department_355_recent_two,department_360_recent_two,department_361_recent_two,department_362_recent_two,department

In [26]:
df_crm_1=pd.read_sql("select * from table_pred_1_crm_up_to_20201226 limit 10;",con=sql_engine)
df_crm_1


,customer_id_hashed,sign_up_location,customer_zip_code,sign_up_date,P_zip,S_zip,else_10_zip,signed_online,email_unsub_label,nearest_BL_store,nearest_BL_dist
0,068e4c4212d500442c039e31ab1bcdf97db2a635fe3d74...,4148,77588,2020-12-26,0,0,0,0,0,1663,0.740126
1,55015ee381d84a126e3cda756d6decc9302f61a912fb22...,1448,18018,2020-12-26,1,1,0,0,0,1380,1.322500
2,cc87c79d1d3f57283fb1dd0940c89a4232ee4a1da740e5...,1789,11433,2020-12-26,0,0,0,0,0,1789,9.441840
3,93661c09b7c8a14af8890ef2e9318e5dc81ac46e9cbeb7...,387,48082,2020-12-26,0,1,0,0,0,5107,2.345210
4,1ba7d1e0bdff93c8e4363f7d4870df61486c7f9818fbcc...,1789,11422,2020-12-26,0,1,0,0,0,1789,8.134770
5,ce553700a680b5883d6f672d3a16db54838708f61cfc57...,556,33612,2020-12-26,1,1,0,0,0,556,2.510210
6,86c52de92c35c7b686a4077d243f1f882e1afc1f4b04bd...,1144,74436,2020-12-26,0,0,0,0,0,1144,16.663800
7,c025db60af07a61dc244459524028ae9aecdc52e9294e3...,4474,83815,2020-12-26,1,0,0,0,0,4474,1.393020
8,20802834ff664cfaad8ae4a20120d6b023c66a593a9705...,4052,90621,2020-12-26,1,1,0,0,0,4052,3.067350
9,023f24a3410f38aab543b77bea2050c50e1294525c4565...,1644,49776,2020-12-26,0,1,0,0,0,1644,18.154800


In [27]:
os.getcwd()

'/mnt/clients/juba/hqjubaapp02/jliang/Projects/Big_Lots/Predictive_Model'

In [29]:
df_result=pd.read_csv("/mnt/clients/juba/hqjubaapp02/jliang/Projects/Big_Lots/Predictive_Model/Model_Outputs/output_LastWeek_NoDCM_2020-12-26_2021-01-04/df_target_ids_labeled_2020-12-12_trans_2_plus_2020-12-26.csv",
                     nrows=20)
df_result

,customer_id_hashed,y_hat,selection_label,sign_up_label,email_subscription_label
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,0.115689,nonselect,existing,default_subs
1,000003066bafc6b19c55d3be5fc646a253c659e9e6baea...,0.145278,target,existing,default_subs
2,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,0.380914,target,existing,default_subs
3,000006aa196c7510ca3b4617e7e52d9527e0190e9152fb...,0.385860,target,existing,default_subs
4,000006e85aba854de9992437786bddfbd409ddbebeeb80...,0.029834,nonselect,existing,default_subs
5,000006ff76d207f0c8fbb0262fb87c64b31a75dc89fd07...,0.093461,nonselect,existing,default_subs
6,000008146a15884db40b753f8829c6887be71e51368303...,0.156787,target,existing,default_subs
7,000008374518306e87838f387eedc47d49a447707dc6da...,0.512039,target,existing,default_subs
8,0000089103600e124943b7df1c8068baefed3386fcdaf0...,0.145272,target,existing,default_subs
9,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,0.296018,target,existing,default_subs
